# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U -q PyDrive
!pip install httplib2==0.15.0
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.files import GoogleDriveFileList
from google.colab import auth
from oauth2client.client import GoogleCredentials

from getpass import getpass
import urllib

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Cloning CLIPPER to access modules.
if 'CLIPPER' not in os.listdir():
    cmd_string = 'git clone https://github.com/PAL-ML/CLIPPER.git'
    os.system(cmd_string)

# Installation

## Install multi label metrics dependencies

In [4]:
! pip install scikit-learn==0.24

     |████████████████████████████████| 22.3MB 135kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


## Install CLIP dependencies

In [5]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 11.0


In [ ]:
! pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████▋            | 709.4MB 1.4MB/s eta 0:05:27

In [ ]:
! pip install ftfy regex
! wget https://openaipublic.azureedge.net/clip/bpe_simple_vocab_16e6.txt.gz -O bpe_simple_vocab_16e6.txt.gz

In [ ]:
!pip install git+https://github.com/Sri-vatsa/CLIP # using this fork because of visualization capabilities

## Install clustering dependencies

In [ ]:
!pip -q install umap-learn>=0.3.7

## Install dataset manager dependencies

In [ ]:
!pip install wget

# Imports

In [ ]:
# ML Libraries
import tensorflow as tf
import tensorflow_hub as hub
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import keras

# Data processing
import PIL
import base64
import imageio
import pandas as pd
import numpy as np
import json

from PIL import Image
import cv2
import imgaug.augmenters as iaa

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
from matplotlib import cm

# Models
import clip

# Datasets
import tensorflow_datasets as tfds

# Misc
import progressbar
import logging
from abc import ABC, abstractmethod
import time
import urllib.request
import os
import itertools
from tqdm import tqdm

# Modules
from CLIPPER.code.ExperimentModules import embedding_models
from CLIPPER.code.ExperimentModules import simclr_data_augmentations
from CLIPPER.code.ExperimentModules.dataset_manager import DatasetManager
from CLIPPER.code.ExperimentModules.clip_few_shot import CLIPFewShotClassifier
from CLIPPER.code.ExperimentModules.utils import (save_npy, load_npy, 
                                                       get_folder_id, 
                                                       create_expt_dir, 
                                                       save_to_drive, 
                                                       load_all_from_drive_folder, 
                                                       download_file_by_name, 
                                                       delete_file_by_name)

logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)

# Initialization & Constants

**Edited**

In [ ]:
dataset_name = "ISR"
folder_name = "IndoorSceneRecognition-Embeddings-28-02-21"

# Change parentid to match that of experiments root folder in gdrive
parentid = '1bK72W-Um20EQDEyChNhNJthUNbmoSEjD'

# Filepaths
train_labels_filename = "train_labels.npz"
val_labels_filename = "val_labels.npz"

train_embeddings_filename_suffix = "_embeddings_train.npz"
val_embeddings_filename_suffix = "_embeddings_val.npz"


# Initialize sepcific experiment folder in drive
folderid = create_expt_dir(drive, parentid, folder_name)

# Load data

In [ ]:
def get_ndarray_from_drive(drive, folderid, filename):
    download_file_by_name(drive, folderid, filename)
    return np.load(filename)['data']

# train_labels = get_ndarray_from_drive(drive, folderid, train_labels_filename)
val_labels = get_ndarray_from_drive(drive, folderid, val_labels_filename)
# test_labels = get_ndarray_from_drive(drive, folderid, test_labels_filename)

In [ ]:
dm = DatasetManager()

test_data_generator = dm.load_dataset('indoor_scene_recognition', split='val')

# Label dicts

In [ ]:
classes = dm.get_class_names()

class_names = [x.replace("_", " ") for x in classes]

# Create label dictionary

In [ ]:
unique_labels = np.unique(val_labels)
print(len(unique_labels))

In [ ]:
label_dictionary = {la:[] for la in unique_labels}

for i in range(len(val_labels)):
    la = val_labels[i]

    label_dictionary[la].append(i)

In [ ]:
print(label_dictionary)

# CLIP zero shot eval

## Function definitions

In [ ]:
def start_progress_bar(bar_len):
    widgets = [
        ' [', 
        progressbar.Timer(format= 'elapsed time: %(elapsed)s'), 
        '] ', 
        progressbar.Bar('*'),' (', 
        progressbar.ETA(), ') ', 
        ]
    pbar = progressbar.ProgressBar(
        max_value=bar_len, widgets=widgets
        ).start()

    return pbar

In [ ]:
def prepare_indices(
    num_ways,
    num_shot,
    num_eval,
    num_episodes,
    label_dictionary,
    test_labels,
    shuffle=False
):
    eval_indices = []
    train_indices = []
    wi_y = []
    eval_y = []

    label_dictionary = {la:label_dictionary[la] for la in label_dictionary if len(label_dictionary[la]) >= (num_shot+num_eval)}
    unique_labels = list(label_dictionary.keys())

    pbar = start_progress_bar(num_episodes)

    for s in range(num_episodes):
        # Setting random seed for replicability
        np.random.seed(s)

        _train_indices = []
        _eval_indices = []

        selected_labels = np.random.choice(unique_labels, size=num_ways, replace=False)
        for la in selected_labels:
            la_indices = label_dictionary[la]
            select = np.random.choice(la_indices, size = num_shot+num_eval, replace=False)
            tr_idx = list(select[:num_shot])
            ev_idx = list(select[num_shot:])

            _train_indices = _train_indices + tr_idx
            _eval_indices = _eval_indices + ev_idx

        if shuffle:
            np.random.shuffle(_train_indices)
            np.random.shuffle(_eval_indices)

        train_indices.append(_train_indices)
        eval_indices.append(_eval_indices)

        _wi_y = test_labels[_train_indices]
        _eval_y = test_labels[_eval_indices]

        wi_y.append(_wi_y)
        eval_y.append(_eval_y)

        pbar.update(s+1)
        
    return train_indices, eval_indices, wi_y, eval_y

In [ ]:
def embed_images(
    embedding_model, 
    train_indices, 
    num_augmentations,
    trivial=False
    ):

    def augment_image(image, num_augmentations, trivial):
        """ Perform SimCLR augmentations on the image
        """

        if np.max(image) > 1:
            image = image/255

        augmented_images = [image]

        # augmentations = iaa.Sequential([
        #     iaa.Affine(
        #         translate_percent={'x':(-0.1, 0.1), 'y':(-0.1, 0.1)}, 
        #         rotate=(-15, 15),
        #         shear=(-15, 15),
        #     ),
        #     iaa.Fliplr(0.5) 
        # ])

        def _run_filters(image):
            width = image.shape[1]
            height = image.shape[0]
            image_aug = simclr_data_augmentations.random_crop_with_resize(
                image,
                height,
                width
                )
            image_aug = tf.image.random_flip_left_right(image_aug)
            image_aug = simclr_data_augmentations.random_color_jitter(image_aug)
            image_aug = simclr_data_augmentations.random_blur(
                image_aug,
                height,
                width
                )
            image_aug = tf.reshape(image_aug, [image.shape[0], image.shape[1], 3])
            image_aug = tf.clip_by_value(image_aug, 0., 1.)

            return image_aug.numpy()

        for _ in range(num_augmentations):
            # aug_image = augmentations(image=image)

            if trivial:
                aug_image = image
            else:
                aug_image = _run_filters(image)
            augmented_images.append(aug_image)

        augmented_images = np.stack(augmented_images)
        return augmented_images

    embedding_model.load_model()

    unique_indices = np.unique(np.array(train_indices))

    ds = dm.load_dataset('indoor_scene_recognition', split='val')
    embeddings = []
    IMAGE_IDX = 0
    pbar = start_progress_bar(unique_indices.size+1)
    num_done=0
    for idx, item in enumerate(ds):
        if idx in unique_indices:
            image = item[IMAGE_IDX][0]
            if num_augmentations > 0:
                aug_images = augment_image(image, num_augmentations, trivial)
            else:
                aug_images = image
            
            processed_images = embedding_model.preprocess_data(aug_images)
            embedding = embedding_model.embed_images(processed_images)
            embeddings.append(embedding)
            
            num_done += 1
            pbar.update(num_done+1)

        if idx == unique_indices[-1]:
            break

    embeddings = np.stack(embeddings)

    return unique_indices, embeddings

In [ ]:
def evaluate_model_for_episode(
    model,
    eval_x,
    eval_y,
    label_mapping,
    filtered_classes,
    metrics=['hamming', 'jaccard', 'subset_accuracy', 'ap', 'map', 'c_f1', 'o_f1', 'c_precision', 'o_precision', 'c_recall', 'o_recall', 'top1_accuracy', 'top5_accuracy', 'classwise_accuracy', 'c_accuracy'],
    multi_label=True
):
    zs_weights = model.zeroshot_classifier(filtered_classes)
    logits = model.predict_scores(eval_x, zs_weights).tolist()
    
    pred_y = model.predict_label(eval_x, zs_weights)
    pred_y = [label_mapping[l] for l in pred_y]

    met = model.evaluate_single_label_metrics(
            eval_x, eval_y, label_mapping, zs_weights, metrics
            )

    return pred_y, met, logits

In [ ]:
def get_label_mapping_n_class_names(eval_y, class_names):
  label_mapping = {}
  unique_labels = np.unique(eval_y)
  filtered_classes = [class_names[x] for x in unique_labels]

  num_classes = len(unique_labels)
  for c in range(num_classes):
    label_mapping[c] = unique_labels[c]

  return label_mapping, filtered_classes

In [ ]:
# chenni change
def run_episode_through_model(
    indices_and_embeddings,
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y,
    class_names,
    num_augmentations=0,
    train_epochs=None,
    train_batch_size=5,
    metrics=['hamming', 'jaccard', 'subset_accuracy', 'ap', 'map', 'c_f1', 'o_f1', 'c_precision', 'o_precision', 'c_recall', 'o_recall', 'top1_accuracy', 'top5_accuracy', 'classwise_accuracy', 'c_accuracy'],
    embeddings=None,
    multi_label=True
):
    metrics_values = {m:[] for m in metrics}
    indices_and_embeddings
    eval_x = embeddings[eval_indices]
    ep_logits = []
    
    label_mapping, filtered_classes = get_label_mapping_n_class_names(eval_y, class_names)

    clip_fs_parameters = {
        "num_classes": num_ways,
        "input_dims": eval_x.shape[-1],
        "multi_label": multi_label
    }

    clip_fs_cls = CLIPFewShotClassifier(clip_fs_parameters)
    
    pred_labels, metrics_values, logits = evaluate_model_for_episode(
            clip_fs_cls,
            eval_x,
            eval_y,
            label_mapping,
            filtered_classes,
            metrics=metrics,
            multi_label=False
    )
    ep_logits = logits
#cc
    return metrics_values, ep_logits

In [ ]:
def run_evaluations(
    indices_and_embeddings,
    train_indices, 
    eval_indices, 
    wi_y, 
    eval_y, 
    num_episodes, 
    num_ways,
    class_names,
    verbose=True,
    normalize=True,
    train_epochs=None,
    train_batch_size=5,
    metrics=['accuracy', 'ap', 'map', 'c_f1', 'o_f1', 'c_precision', 'o_precision', 'c_recall', 'o_recall', 'top1_accuracy', 'top5_accuracy', 'classwise_accuracy', 'c_accuracy'],
    embeddings=None,
    num_augmentations=0,
    multi_label=True
):
    metrics_values = {m:[] for m in metrics}

    all_logits = []

    if verbose:
        pbar = start_progress_bar(num_episodes)

    for idx_ep in range(num_episodes):
        _train_indices = train_indices[idx_ep]
        _eval_indices = eval_indices[idx_ep]
        _wi_y = [label for label in wi_y[idx_ep]]
        _eval_y = [label for label in eval_y[idx_ep]]

        met, ep_logits = run_episode_through_model(
            indices_and_embeddings,
            _train_indices, 
            _eval_indices, 
            _wi_y, 
            _eval_y,
            class_names,
            num_augmentations=num_augmentations,
            train_epochs=train_epochs,
            train_batch_size=train_batch_size,
            embeddings=embeddings,
            metrics=metrics,
            multi_label=multi_label
        )

        all_logits.append(ep_logits)

        for m in metrics:
            metrics_values[m].append(met[m])

        if verbose:
            pbar.update(idx_ep+1)

    return metrics_values, all_logits

In [ ]:
def get_best_metric(mt, metric_name, optimal='max'):
    if optimal=='max':
        opt_value = np.max(np.mean(np.array(mt[metric_name]), axis=0))
    if optimal=='min':
        opt_value = np.min(np.mean(np.array(mt[metric_name]), axis=0))

    return opt_value

# 5 way 5 shot

## Picking indices

In [ ]:
num_ways = 5
num_shot = 5
num_eval = 15
shuffle = False
num_episodes = 100

train_indices, eval_indices, wi_y, eval_y = prepare_indices(
    num_ways, num_shot, num_eval, num_episodes, label_dictionary, val_labels, shuffle
)

In [ ]:
embedding_model = embedding_models.CLIPEmbeddingWrapper()
num_augmentations = 0
trivial=False

indices, embeddings = embed_images(
    embedding_model, 
    train_indices, 
    num_augmentations,
    trivial=trivial
    )

## CLIP

In [ ]:
clip_embeddings_test_fn = "clip" + val_embeddings_filename_suffix
clip_embeddings_test = get_ndarray_from_drive(drive, folderid, clip_embeddings_test_fn)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

if trivial:
  #cc
    results_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_clip_zs_metrics_with_logits.json"
else:
  #cc
    results_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_clip_zs_metrics_with_logits.json"

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download_file_by_name(drive, folderid, results_filename)
if results_filename in os.listdir():
    with open(results_filename, 'r') as f:
      #cc
        json_loaded = json.load(f)
        #cc
        clip_metrics_over_train_epochs = json_loaded['metrics']
        #cc
        logits_over_train_epochs = json_loaded["logits"]
else:
    clip_metrics_over_train_epochs = []  
    logits_over_train_epochs = []   

train_epochs_arr = [0]
multi_label=False
# metrics_vals = ['hamming', 'jaccard', 'f1_score'] # ['accuracy', 'f1_score']

for idx, train_epochs in enumerate(train_epochs_arr):
    if idx < len(clip_metrics_over_train_epochs):
        continue
    print(train_epochs)
    #cc
    clip_metrics_thresholds, all_logits = run_evaluations(
        (indices, embeddings),
        train_indices,
        eval_indices,
        wi_y,
        eval_y,
        num_episodes,
        num_ways,
        class_names,
        train_epochs=train_epochs,
        num_augmentations=num_augmentations,
        embeddings=clip_embeddings_test
    )
    clip_metrics_over_train_epochs.append(clip_metrics_thresholds)
    #cc
    logits_over_train_epochs.append(all_logits)

#cc
    fin_list = []
    #cc the whole for loop
    for a1 in wi_y:
      fin_a1_list = []
      for a2 in a1:
        new_val = str(a2)
        fin_a1_list.append(new_val)
      fin_list.append(fin_a1_list)

    with open(results_filename, 'w') as f:
      #cc
        results = {'metrics': clip_metrics_over_train_epochs,
                   "logits": logits_over_train_epochs,
                   "true_labels": fin_list}
        json.dump(results, f)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    #delete_file_by_name(drive, folderid, results_filename)
    #save_to_drive(drive, folderid, results_filename)

In [ ]:
if trivial:
    PLOT_DIR = "NewMetrics_clip_zs_Sigmoid_IndoorSceneRecognition" + "_0t" + str(num_ways) + "w" + str(num_shot) + "s" + str(num_augmentations) + "a_trivial_plots"
else:
    PLOT_DIR = "NewMetrics_clip_zs_Sigmoid_IndoorSceneRecognition" + "_0t" + str(num_ways) + "w" + str(num_shot) + "s" + str(num_augmentations) + "a_plots"
os.mkdir(PLOT_DIR)

In [ ]:
# chenni change whole block
all_metrics = ['accuracy', 'ap', 'map', 'c_f1', 'o_f1', 'c_precision', 'o_precision', 'c_recall', 'o_recall', 'top1_accuracy', 'top5_accuracy', 'classwise_accuracy', 'c_accuracy']

final_dict = {}
for ind_metric in all_metrics:
  vals = []
  final_array = []
  for mt in clip_metrics_over_train_epochs:
      ret_val = get_best_metric(mt,ind_metric,"max")
      vals.append(ret_val)

  final_array.append(vals)
  final_dict[ind_metric] = final_array

if trivial:
    graph_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_clip_zs_metrics_graphs.json"
else:
    graph_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_clip_zs_metrics_graphs.json"

with open(graph_filename, 'w') as f:
        json.dump(final_dict, f)

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
delete_file_by_name(drive, folderid, graph_filename)
save_to_drive(drive, folderid, graph_filename)

In [ ]:
zip_dirname = PLOT_DIR + ".zip"
zip_source = PLOT_DIR
! zip -r $zip_dirname $zip_source

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
save_to_drive(drive, folderid, zip_dirname)

# 13 way 5 shot

## Picking indices

In [ ]:
num_ways = 13
num_shot = 5
num_eval = 5
shuffle = False
num_episodes = 100

train_indices, eval_indices, wi_y, eval_y = prepare_indices(
    num_ways, num_shot, num_eval, num_episodes, label_dictionary, val_labels, shuffle
)

In [ ]:
embedding_model = embedding_models.CLIPEmbeddingWrapper()
num_augmentations = 0
trivial=False

indices, embeddings = embed_images(
    embedding_model, 
    train_indices, 
    num_augmentations,
    trivial=trivial
    )

## CLIP

In [ ]:
clip_embeddings_test_fn = "clip" + val_embeddings_filename_suffix
clip_embeddings_test = get_ndarray_from_drive(drive, folderid, clip_embeddings_test_fn)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

if trivial:
  #cc
    results_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_clip_zs_metrics_with_logits.json"
else:
  #cc
    results_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_metrics_clip_zs_with_logits.json"

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download_file_by_name(drive, folderid, results_filename)
if results_filename in os.listdir():
    with open(results_filename, 'r') as f:
      #cc
        json_loaded = json.load(f)
        #cc
        clip_metrics_over_train_epochs = json_loaded['metrics']
        #cc
        logits_over_train_epochs = json_loaded["logits"]
else:
    clip_metrics_over_train_epochs = []  
    #cc  
    logits_over_train_epochs = []   

train_epochs_arr = [0]
multi_label=False
# metrics_vals = ['hamming', 'jaccard', 'f1_score'] # ['accuracy', 'f1_score']

for idx, train_epochs in enumerate(train_epochs_arr):
    if idx < len(clip_metrics_over_train_epochs):
        continue
    print(train_epochs)
    #cc
    clip_metrics_thresholds, all_logits = run_evaluations(
        (indices, embeddings),
        train_indices,
        eval_indices,
        wi_y,
        eval_y,
        num_episodes,
        num_ways,
        class_names,
        train_epochs=train_epochs,
        num_augmentations=num_augmentations,
        embeddings=clip_embeddings_test
    )
    clip_metrics_over_train_epochs.append(clip_metrics_thresholds)
    #cc
    logits_over_train_epochs.append(all_logits)

#cc
    fin_list = []
    #cc the whole for loop
    for a1 in wi_y:
      fin_a1_list = []
      for a2 in a1:
        new_val = str(a2)
        fin_a1_list.append(new_val)
      fin_list.append(fin_a1_list)

    with open(results_filename, 'w') as f:
      #cc
        results = {'metrics': clip_metrics_over_train_epochs,
                   "logits": logits_over_train_epochs,
                   "true_labels": fin_list}
        json.dump(results, f)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    delete_file_by_name(drive, folderid, results_filename)
    save_to_drive(drive, folderid, results_filename)

In [ ]:
if trivial:
    PLOT_DIR = "NewMetrics_clip_zs_Sigmoid_IndoorSceneRecognition" + "_0t" + str(num_ways) + "w" + str(num_shot) + "s" + str(num_augmentations) + "a_trivial_plots"
else:
    PLOT_DIR = "NewMetrics_clip_zs_Sigmoid_IndoorSceneRecognition" + "_0t" + str(num_ways) + "w" + str(num_shot) + "s" + str(num_augmentations) + "a_plots"
os.mkdir(PLOT_DIR)

In [ ]:
all_metrics = ['accuracy', 'ap', 'map', 'c_f1', 'o_f1', 'c_precision', 'o_precision', 'c_recall', 'o_recall', 'top1_accuracy', 'top5_accuracy', 'classwise_accuracy', 'c_accuracy']

final_dict = {}
for ind_metric in all_metrics:
  vals = []
  final_array = []
  for mt in clip_metrics_over_train_epochs:
      ret_val = get_best_metric(mt,ind_metric,"max")
      vals.append(ret_val)

  final_array.append(vals)
  final_dict[ind_metric] = final_array

if trivial:
    graph_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_clip_zs_metrics_graphs.json"
else:
    graph_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_clip_zs_metrics_graphs.json"

with open(graph_filename, 'w') as f:
        json.dump(final_dict, f)

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
delete_file_by_name(drive, folderid, graph_filename)
save_to_drive(drive, folderid, graph_filename)

In [ ]:
zip_dirname = PLOT_DIR + ".zip"
zip_source = PLOT_DIR
! zip -r $zip_dirname $zip_source

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
save_to_drive(drive, folderid, zip_dirname)

# 5 way 1 shot

## Picking indices

In [ ]:
num_ways = 5
num_shot = 1
num_eval = 19
shuffle = False
num_episodes = 100

train_indices, eval_indices, wi_y, eval_y = prepare_indices(
    num_ways, num_shot, num_eval, num_episodes, label_dictionary, val_labels, shuffle
)

In [ ]:
embedding_model = embedding_models.CLIPEmbeddingWrapper()
num_augmentations = 0
trivial=False

indices, embeddings = embed_images(
    embedding_model, 
    train_indices, 
    num_augmentations,
    trivial=trivial
    )

## CLIP

In [ ]:
clip_embeddings_test_fn = "clip" + val_embeddings_filename_suffix
clip_embeddings_test = get_ndarray_from_drive(drive, folderid, clip_embeddings_test_fn)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

if trivial:
  #cc
    results_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_clip_zs_metrics_with_logits.json"
else:
  #cc
    results_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_clip_zs_metrics_with_logits.json"

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download_file_by_name(drive, folderid, results_filename)
if results_filename in os.listdir():
    with open(results_filename, 'r') as f:
      #cc
        json_loaded = json.load(f)
        #cc
        clip_metrics_over_train_epochs = json_loaded['metrics']
        #cc
        logits_over_train_epochs = json_loaded["logits"]
else:
    clip_metrics_over_train_epochs = []  
    #cc  
    logits_over_train_epochs = []   

train_epochs_arr = [0]
multi_label=False
# metrics_vals = ['hamming', 'jaccard', 'f1_score'] # ['accuracy', 'f1_score']

for idx, train_epochs in enumerate(train_epochs_arr):
    if idx < len(clip_metrics_over_train_epochs):
        continue
    print(train_epochs)
    #cc
    clip_metrics_thresholds, all_logits = run_evaluations(
        (indices, embeddings),
        train_indices,
        eval_indices,
        wi_y,
        eval_y,
        num_episodes,
        num_ways,
        class_names,
        train_epochs=train_epochs,
        num_augmentations=num_augmentations,
        embeddings=clip_embeddings_test
    )
    clip_metrics_over_train_epochs.append(clip_metrics_thresholds)
    #cc
    logits_over_train_epochs.append(all_logits)

#cc
    fin_list = []
    #cc the whole for loop
    for a1 in wi_y:
      fin_a1_list = []
      for a2 in a1:
        new_val = str(a2)
        fin_a1_list.append(new_val)
      fin_list.append(fin_a1_list)

    with open(results_filename, 'w') as f:
      #cc
        results = {'metrics': clip_metrics_over_train_epochs,
                   "logits": logits_over_train_epochs,
                   "true_labels": fin_list}
        json.dump(results, f)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    delete_file_by_name(drive, folderid, results_filename)
    save_to_drive(drive, folderid, results_filename)

In [ ]:
if trivial:
    PLOT_DIR = "NewMetrics_clip_zs_Sigmoid_IndoorSceneRecognition" + "_0t" + str(num_ways) + "w" + str(num_shot) + "s" + str(num_augmentations) + "a_trivial_plots"
else:
    PLOT_DIR = "NewMetrics_clip_zs_Sigmoid_IndoorSceneRecognition" + "_0t" + str(num_ways) + "w" + str(num_shot) + "s" + str(num_augmentations) + "a_plots"
os.mkdir(PLOT_DIR)

In [ ]:
all_metrics = ['accuracy', 'ap', 'map', 'c_f1', 'o_f1', 'c_precision', 'o_precision', 'c_recall', 'o_recall', 'top1_accuracy', 'top5_accuracy', 'classwise_accuracy', 'c_accuracy']

final_dict = {}
for ind_metric in all_metrics:
  vals = []
  final_array = []
  for mt in clip_metrics_over_train_epochs:
      ret_val = get_best_metric(mt,ind_metric,"max")
      vals.append(ret_val)

  final_array.append(vals)
  final_dict[ind_metric] = final_array

if trivial:
    graph_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_clip_zs_metrics_graphs.json"
else:
    graph_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_clip_zs_metrics_graphs.json"

with open(graph_filename, 'w') as f:
        json.dump(final_dict, f)

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
delete_file_by_name(drive, folderid, graph_filename)
save_to_drive(drive, folderid, graph_filename)

In [ ]:
zip_dirname = PLOT_DIR + ".zip"
zip_source = PLOT_DIR
! zip -r $zip_dirname $zip_source

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
save_to_drive(drive, folderid, zip_dirname)

# 13 way 1 shot

## Picking indices

In [ ]:
num_ways = 13
num_shot = 1
num_eval = 10
shuffle = False
num_episodes = 100

train_indices, eval_indices, wi_y, eval_y = prepare_indices(
    num_ways, num_shot, num_eval, num_episodes, label_dictionary, val_labels, shuffle
)

In [ ]:
embedding_model = embedding_models.CLIPEmbeddingWrapper()
num_augmentations = 0
trivial=False

indices, embeddings = embed_images(
    embedding_model, 
    train_indices, 
    num_augmentations,
    trivial=trivial
    )

## CLIP

In [ ]:
clip_embeddings_test_fn = "clip" + val_embeddings_filename_suffix
clip_embeddings_test = get_ndarray_from_drive(drive, folderid, clip_embeddings_test_fn)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

if trivial:
  #cc
    results_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_clip_zs_metrics_with_logits.json"
else:
  #cc
    results_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_clip_zs_metrics_with_logits.json"

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download_file_by_name(drive, folderid, results_filename)
if results_filename in os.listdir():
    with open(results_filename, 'r') as f:
      #cc
        json_loaded = json.load(f)
        #cc
        clip_metrics_over_train_epochs = json_loaded['metrics']
        #cc
        logits_over_train_epochs = json_loaded["logits"]
else:
    clip_metrics_over_train_epochs = []  
    #cc  
    logits_over_train_epochs = []   

train_epochs_arr = [0]
multi_label=False
# metrics_vals = ['hamming', 'jaccard', 'f1_score'] # ['accuracy', 'f1_score']

for idx, train_epochs in enumerate(train_epochs_arr):
    if idx < len(clip_metrics_over_train_epochs):
        continue
    print(train_epochs)
    #cc
    clip_metrics_thresholds, all_logits = run_evaluations(
        (indices, embeddings),
        train_indices,
        eval_indices,
        wi_y,
        eval_y,
        num_episodes,
        num_ways,
        class_names,
        train_epochs=train_epochs,
        num_augmentations=num_augmentations,
        embeddings=clip_embeddings_test
    )
    clip_metrics_over_train_epochs.append(clip_metrics_thresholds)
    #cc
    logits_over_train_epochs.append(all_logits)

#cc
    fin_list = []
    #cc the whole for loop
    for a1 in wi_y:
      fin_a1_list = []
      for a2 in a1:
        new_val = str(a2)
        fin_a1_list.append(new_val)
      fin_list.append(fin_a1_list)

    with open(results_filename, 'w') as f:
      #cc
        results = {'metrics': clip_metrics_over_train_epochs,
                   "logits": logits_over_train_epochs,
                   "true_labels": fin_list}
        json.dump(results, f)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    delete_file_by_name(drive, folderid, results_filename)
    save_to_drive(drive, folderid, results_filename)

In [ ]:
if trivial:
    PLOT_DIR = "NewMetrics_clip_zs_Sigmoid_IndoorSceneRecognition" + "_0t" + str(num_ways) + "w" + str(num_shot) + "s" + str(num_augmentations) + "a_trivial_plots"
else:
    PLOT_DIR = "NewMetrics_clip_zs_Sigmoid_IndoorSceneRecognition" + "_0t" + str(num_ways) + "w" + str(num_shot) + "s" + str(num_augmentations) + "a_plots"
os.mkdir(PLOT_DIR)

In [ ]:
all_metrics = ['accuracy', 'ap', 'map', 'c_f1', 'o_f1', 'c_precision', 'o_precision', 'c_recall', 'o_recall', 'top1_accuracy', 'top5_accuracy', 'classwise_accuracy', 'c_accuracy']

final_dict = {}
for ind_metric in all_metrics:
  vals = []
  final_array = []
  for mt in clip_metrics_over_train_epochs:
      ret_val = get_best_metric(mt,ind_metric,"max")
      vals.append(ret_val)

  final_array.append(vals)
  final_dict[ind_metric] = final_array

if trivial:
    graph_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_trivial_clip_zs_metrics_graphs.json"
else:
    graph_filename = "new_metrics"+dataset_name+"_0t"+str(num_ways)+"w"+str(num_shot)+"s"+str(num_augmentations)+"a_clip_zs_metrics_graphs.json"

with open(graph_filename, 'w') as f:
        json.dump(final_dict, f)

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
delete_file_by_name(drive, folderid, graph_filename)
save_to_drive(drive, folderid, graph_filename)

In [ ]:
zip_dirname = PLOT_DIR + ".zip"
zip_source = PLOT_DIR
! zip -r $zip_dirname $zip_source

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
save_to_drive(drive, folderid, zip_dirname)